## Imports

In [13]:

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("Loads raw to ref").getOrCreate()

Lendo o arquivo

In [14]:
df = spark.read.csv('04112022.csv', header=True, inferSchema=True, sep="\u0001")

Função UDF para verificar se o primeiro emoji encontrado é positivo ou negativo, retorna uma string('positivo','negativo' ou 'erro')

In [15]:
def check_first(text):
    pos_emojis = [':D',':)',':]',':}']
    neg_emojis = [':c',':(',':[',':{']
    index_pos = 1000
    index_neg = 1000
    for emoji_pos in pos_emojis:
        temp = text.find(emoji_pos)
        if temp < index_pos and temp != -1:
            index_pos = temp
    for emoji_neg in neg_emojis:
        temp = text.find(emoji_neg)
        if temp < index_neg and temp != -1:
            index_neg = temp
    result = 'positivo' if index_neg > index_pos and index_neg != 1000 else('negativo' if index_neg < index_pos and index_pos != 1000 else 'erro')
    return result

checar = udf(check_first, StringType())

Retorna uma string contendo o primeiro emoji encontrado

In [16]:
def get_emogi(text):
    emojis = [':D',':)',':]',':}',':c',':(',':[',':{']
    index = 1000
    emoji_r = ''
    for emoji in emojis:
        temp = text.find(emoji)
        if temp < index and temp != -1:
            emoji_r = emoji
            index = temp
    return emoji_r 

getter = udf(get_emogi, StringType())

Define os emojis negativos e positivos a serem considerados

In [17]:
regex_pos = ":\)|:D|:\]|:\}"
regex_neg = ":\[|:c|:\(|:\{"

In [18]:

df = df.select('*').withColumn(
    'sentimento', when(
        (col("tweet_text").rlike(regex_pos)) & (col("tweet_text").rlike(regex_neg)), checar('tweet_text')
    ).when(
        (col("tweet_text").rlike(regex_pos)) & ~(col("tweet_text").rlike(regex_neg)), 'positivo'
    ).when(
        ~(col("tweet_text").rlike(regex_pos)) & (col("tweet_text").rlike(regex_neg)), 'negativo'
    ).when(
        ~(col("tweet_text").rlike(regex_pos)) & ~(col("tweet_text").rlike(regex_neg)), 'neutro'
    )
).withColumn('emoji',
    when(
       col('sentimento') == 'neutro', 'n/a' 
    ).otherwise(
        getter('tweet_text')
    )
)


Limpando bad data detectado

In [19]:
df = df.filter(
    (df['tweet_date'] != '0002-12-31 00:00:00')
)

Criando colunas para particionamento

In [20]:

df_final = df.withColumn("timestamp", to_timestamp(col("tweet_date"), 'yyyyMMddHHmm')) \
           .withColumn("year", date_format(col("timestamp"), "yyyy")) \
           .withColumn("month", date_format(col("timestamp"), "MM")) \
           .withColumn("day", date_format(col("timestamp"), "dd")) \
           .drop("timestamp")

Escrevendo dados

In [21]:
df_final.write.partitionBy("year", "month", "day") \
    .mode("overwrite") \
    .option("header", "false").option("sep", "\u0001") \
    .csv('teste')